In [ ]:
import json
import pandas as pd

from markdownify import markdownify

from datetime import date, datetime, time
from backports.datetime_fromisoformat import MonkeyPatch

In [ ]:
MonkeyPatch.patch_fromisoformat()

Use this url to export data

https://www.blogger.com/feeds/{{blog_id}}/posts/default?alt=json&orderby=published&sortorder=ascending&max-results=1000

In [ ]:
data = json.load(open("blog.json", "r"))

In [ ]:
entries = data["feed"]["entry"]

In [ ]:
posts = []
for entry in entries:
    
    title = entry["title"]["$t"]
    content = entry["content"]["$t"]
    
    thumbnail = entry["media$thumbnail"]["url"] if entry.get("media$thumbnail") else None
    
    tags = [c.get("term") for c in entry.get("category", [])]
    
    publish_datetime = entry["published"]["$t"]
    
    url = entry["link"][-1]["href"]
    
    markdown = markdownify(content)
    
    posts.append({
        "title": title,
        "content": content,
        "markdown": markdown,
        "thumbnail": thumbnail,
        "tags": tags,
        "url": url,
        "publish_datetime": publish_datetime,
        
    })
    
    #print(entry)
    #break

In [ ]:
df = pd.DataFrame(posts)

In [ ]:
df

In [ ]:
def generate_mdx(row):

    dt = datetime.fromisoformat(row.publish_datetime, "%Y-%m-%dT%H:%M:%S.%f%z")
    
    lines = ["---"]
    
    lines.append("title: '{}'".format(row.title))
    lines.append("date: '{}'".format(dt.strftime("%Y-%m-%d %H:%M:%S")))
    
    lines.append("---")
    lines.append("")
    
    lines.append(row.markdown)
    
    return "\n".join(lines)

In [ ]:
base_path = "/home/null/git/kittinan.github.io/posts"

for i, row in df.iterrows():
    
    dt = datetime.fromisoformat(row.publish_datetime, "%Y-%m-%dT%H:%M:%S.%f%z")
    
    
    mdx = generate_mdx(row)
    name = row.url.split("/")[-1]

    file_name = "{}-{}".format(dt.strftime("%Y-%m-%d"), name)
    
    save_path = f"{base_path}/{file_name}.md"
    
    with open(save_path, "w") as f:
        f.write(mdx)
